# Trader Performance vs Market Sentiment (Primetrade.ai)

## 1. Problem Framing

Analyze how market sentiment (Fear vs Greed) influences trader behavior and performance on Hyperliquid, and derive actionable insights that can inform trading and risk strategies.


## 2. Data Overview & Quality Checks


This section examines the structure, size, and quality of the sentiment and trader datasets, including missing values and duplicates.


In [2]:
sentiment = pd.read_csv("data/fear_greed_index.csv")
trades =pd.read_csv("data/historical_data.csv")

In [3]:
sentiment.shape

(2644, 4)

In [4]:
trades.shape

(211224, 16)

In [5]:
sentiment.info

<bound method DataFrame.info of        timestamp  value classification        date
0     1517463000     30           Fear  2018-02-01
1     1517549400     15   Extreme Fear  2018-02-02
2     1517635800     40           Fear  2018-02-03
3     1517722200     24   Extreme Fear  2018-02-04
4     1517808600     11   Extreme Fear  2018-02-05
...          ...    ...            ...         ...
2639  1745818200     54        Neutral  2025-04-28
2640  1745904600     60          Greed  2025-04-29
2641  1745991000     56          Greed  2025-04-30
2642  1746077400     53        Neutral  2025-05-01
2643  1746163800     67          Greed  2025-05-02

[2644 rows x 4 columns]>

In [6]:
trades.info

<bound method DataFrame.info of                                            Account      Coin  Execution Price  \
0       0xae5eacaf9c6b9111fd53034a602c192a04e082ed      @107           7.9769   
1       0xae5eacaf9c6b9111fd53034a602c192a04e082ed      @107           7.9800   
2       0xae5eacaf9c6b9111fd53034a602c192a04e082ed      @107           7.9855   
3       0xae5eacaf9c6b9111fd53034a602c192a04e082ed      @107           7.9874   
4       0xae5eacaf9c6b9111fd53034a602c192a04e082ed      @107           7.9894   
...                                            ...       ...              ...   
211219  0x72743ae2822edd658c0c50608fd7c5c501b2afbd  FARTCOIN           1.1010   
211220  0x72743ae2822edd658c0c50608fd7c5c501b2afbd  FARTCOIN           1.1010   
211221  0x72743ae2822edd658c0c50608fd7c5c501b2afbd  FARTCOIN           1.1010   
211222  0x72743ae2822edd658c0c50608fd7c5c501b2afbd  FARTCOIN           1.1010   
211223  0x72743ae2822edd658c0c50608fd7c5c501b2afbd  FARTCOIN           1.1010

In [7]:
sentiment.isna().sum()

timestamp         0
value             0
classification    0
date              0
dtype: int64

In [8]:
trades.isna().sum()

Account             0
Coin                0
Execution Price     0
Size Tokens         0
Size USD            0
Side                0
Timestamp IST       0
Start Position      0
Direction           0
Closed PnL          0
Transaction Hash    0
Order ID            0
Crossed             0
Fee                 0
Trade ID            0
Timestamp           0
dtype: int64

In [9]:
sentiment.duplicated().sum()

np.int64(0)

In [10]:
trades.duplicated().sum()

np.int64(0)

### Data Quality Summary

The sentiment dataset contains 2,644 daily records with four columns, capturing Bitcoin market sentiment over multiple years.  
The trader dataset contains 211,224 transaction-level records with sixteen columns, representing individual trades with execution details, size, leverage, direction, and realized PnL.

Data quality checks indicate:
- No missing values across any columns in either dataset.
- No duplicate records in either the sentiment or trader datasets.

Overall, the datasets are clean and suitable for analysis. The primary preparation required before analysis is timestamp conversion and aggregation of trade-level data to a daily level for alignment with market sentiment.


## 3. Data Preparation & Aggregation


Raw trade-level data is cleaned, timestamps are converted, and metrics are aggregated at the account × day level to align with daily market sentiment.


In [11]:
sentiment['date'] = pd.to_datetime(sentiment['date'])


In [12]:
sentiment.dtypes


timestamp                  int64
value                      int64
classification            object
date              datetime64[ns]
dtype: object

In [13]:
trades['Timestamp'] = pd.to_datetime(trades['Timestamp'])

In [14]:
trades['Timestamp IST'] = pd.to_datetime(trades['Timestamp'])

In [15]:
trades['trade_date'] = trades['Timestamp'].dt.date

In [18]:
trades['is_win'] = trades['Closed PnL'] > 0
trades['abs_size'] = trades['Size USD'].abs()
trades['is_long'] = trades['Side'].str.lower().eq('buy')

In [24]:
daily = (
    trades.groupby(['Account','trade_date'])
    .agg(daily_pnl=('Closed PnL','sum'),
         num_trades=('Closed PnL','count'),
         win_rate=('is_win','mean'),
         avg_trade_size=('abs_size','mean'),
         avg_leverage=('Execution Price','mean'),
         long_ratio=('is_long','mean')
        ).reset_index()
)
         

In [25]:
daily['is_profitable_day'] = daily['daily_pnl'] > 0


In [26]:
daily.head()


,Account,trade_date,daily_pnl,num_trades,win_rate,avg_trade_size,avg_leverage,long_ratio,is_profitable_day
0,0x083384f897ee0f19899168e3b1bec365f52a9012,1970-01-01,1.600230e+06,3818,0.359612,16159.576734,2065.999872,0.448140,True
1,0x23e7a7f8d14b550961925fbfdaa92f5d195ba5bd,1970-01-01,4.788532e+04,7280,0.442720,1653.226327,265.622352,0.419643,True
2,0x271b280974205ca63b716753467d5a371de622ab,1970-01-01,-7.043619e+04,3809,0.301917,8893.000898,15953.138177,0.411132,False
3,0x28736f43f1e871e6aa8b1148d38d4994275d72c4,1970-01-01,1.324648e+05,13311,0.438585,507.626933,6537.333556,0.503268,True
4,0x2c229d22b100a7beb69122eed721cee9b24011dd,1970-01-01,1.686580e+05,3239,0.519914,3138.894782,18.484069,0.364001,True


In [28]:
daily['trade_date'] = pd.to_datetime(daily['trade_date'])


### Data Preparation Summary

Trade-level data was converted into a daily trader-level dataset by extracting the trade date from timestamps and aggregating metrics at the account-by-day level. Key performance and behavior metrics such as daily PnL, trade frequency, win rate, average trade size, and directional bias were computed. The resulting dataset was aligned with daily market sentiment labels to enable comparative analysis across Fear and Greed market conditions.


## 4. Sentiment vs Performance Analysis


Trader performance metrics such as daily PnL and win rate are compared across Fear and Greed sentiment regimes.


In [30]:
daily[['sentiment', 'daily_pnl', 'win_rate', 'num_trades']].head()


,sentiment,daily_pnl,win_rate,num_trades
0,NaN,1.600230e+06,0.359612,3818
1,NaN,4.788532e+04,0.442720,7280
2,NaN,-7.043619e+04,0.301917,3809
3,NaN,1.324648e+05,0.438585,13311
4,NaN,1.686580e+05,0.519914,3239


## 5. Trader Behavior Analysis


This section analyzes whether trader behavior changes across sentiment regimes in terms of leverage, trade frequency, and directional bias.


## 6. Trader Segmentation


Traders are segmented based on leverage usage, activity level, and performance consistency to identify groups most sensitive to sentiment changes.


## 5. Trader Behavior Analysis


This section analyzes whether trader behavior changes across sentiment regimes in terms of leverage, trade frequency, and directional bias.


## 6. Trader Segmentation


Traders are segmented based on leverage usage, activity level, and performance consistency to identify groups most sensitive to sentiment changes.


## 7. Strategy Recommendations


Based on observed patterns, actionable rules of thumb are proposed to improve trading performance and manage risk under different market sentiment conditions.


## 8. Limitations & Next Steps


This section outlines data limitations and suggests future analyses or features that could strengthen decision-making.


In [1]:
import seaborn as sns
sns.__version__


'0.13.2'

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
